In [ ]:
!pip install markdown
!pip install langchain
!pip install pdfminer.six
pip install InstructorEmbedding
pip install sentence-transformers

pip install --upgrade InstructorEmbedding
pip install InstructorEmbedding langchain langchain-community faiss-cpu

pip install InstructorEmbedding sentence-transformers
pip install -U langchain-community
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
pip install InstructorEmbedding

LOADING DATASET


In [1]:
from datasets import load_dataset

ds = load_dataset("rajpurkar/squad")

IMPORTING NECESSARY LIBRARIES


In [4]:
import os
import re
import markdown
from pdfminer.high_level import extract_text as extract_text_from_pdf
from io import StringIO
from html.parser import HTMLParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS


JUST CHECKING AND PRINTING DOC 1 TO SEE IF DATA IS LOADED CORRECTLY


In [1]:
import time
from datasets import load_dataset

# Load the SQuAD dataset
start_time = time.time()
ds = load_dataset("rajpurkar/squad")
print(f"Loaded dataset in {time.time() - start_time:.2f} seconds")

# Select a single document for processing
document_to_process = ds['train'][0]
print("Selected document:", document_to_process['context'])


Loaded dataset in 14.13 seconds
Selected document: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.


PREPROCESSING DATASET 

INITIALIZING MODEL


EMBEDDING AND INDEXING ALL DOCS USING FIASS

In [55]:
import time
import re
from datasets import load_dataset
from langchain.text_splitter import RecursiveCharacterTextSplitter

def clean_markdown(text):
    """Clean Markdown syntax from text."""
    text = re.sub(r'\[([^\]]+)\]\([^)]+\)', r'\1', text)
    text = re.sub(r'\*\*([^*]+)\*\*', r'\1', text)
    text = re.sub(r'\*([^*]+)\*', r'\1', text)
    text = re.sub(r'__([^_]+)__', r'\1', text)
    text = re.sub(r'_([^_]+)_', r'\1', text)
    text = re.sub(r'!\[[^\]]]*]\([^)]*\)', '', text)
    text = re.sub(r'#+\s?', '', text)
    text = re.sub(r'\|', ' ', text)
    text = re.sub(r'-{2,}', '', text)
    text = re.sub(r'\n{2,}', '\n', text)
    return text.strip()  # Strip extra spaces

def process_context(entry, chunk_size, chunk_overlap):
    """Process a single context and return document chunks."""
    context = entry['context']
    clean_context = clean_markdown(context)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.split_text(clean_context)
    
    processed_docs = []
    for j, chunk in enumerate(docs):
        metadata = {
            "Document ID": entry['id'],
            "Chunk Number": j + 1,
        }
        header = f"Document ID: {entry['id']}\n"
        for key, value in metadata.items():
            header += f"{key}: {value}\n"
        chunk_with_header = header + chunk
        processed_docs.append(chunk_with_header)
    
    return processed_docs

# Parameters for text splitting
chunk_size = 1200
chunk_overlap = 100

# Load the SQuAD dataset
start_time = time.time()
ds = load_dataset("rajpurkar/squad")
print(f"Loaded dataset in {time.time() - start_time:.2f} seconds")

# Process only the first 10 documents in the dataset
start_time = time.time()
processed_docs = []
for entry in ds['train'].select(range(50000)):
    processed_docs.extend(process_context(entry, chunk_size, chunk_overlap))
print(f"Processed 10 documents in {time.time() - start_time:.2f} seconds")

from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
start_time = time.time()
model = SentenceTransformer(model_name)
print(f"Initialized model in {time.time() - start_time:.2f} seconds")

# Initialize HuggingFaceEmbeddings
hf_embedding = HuggingFaceEmbeddings(model_name=model_name)

# Embed and index all the documents using FAISS
start_time = time.time()
db = FAISS.from_texts(processed_docs, hf_embedding)
print(f"Created FAISS index in {time.time() - start_time:.2f} seconds")

# Save the indexed data locally
start_time = time.time()
db.save_local("faiss_AiDoc")
print(f"Saved FAISS index in {time.time() - start_time:.2f} seconds")

print("Embeddings and FAISS index created and saved successfully.")


Loaded dataset in 15.62 seconds
Processed 10 documents in 29.46 seconds
Initialized model in 4.81 seconds
Created FAISS index in 4610.99 seconds
Saved FAISS index in 1.04 seconds
Embeddings and FAISS index created and saved successfully.


In [56]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

# Load the model
model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=model_name)
sentence_model = SentenceTransformer(model_name)

# Assume 'processed_docs' contains the processed document texts
embeddings = embedding_model.embed_documents(processed_docs)

# Convert embeddings list to NumPy array
embeddings = np.array(embeddings)

# Initialize FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

# Save the FAISS index
faiss.write_index(index, "faiss_index.bin")

# Save additional data
metadata = {
    "processed_docs": processed_docs,
    "model_name": model_name
}

with open("faiss_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)


In [42]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pickle

# Load FAISS index
index = faiss.read_index("faiss_index.bin")

# Load metadata
with open("faiss_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

processed_docs = metadata["processed_docs"]
model_name = metadata["model_name"]

# Re-initialize the embedding model
embedding_model = HuggingFaceEmbeddings(model_name=model_name)
sentence_model = SentenceTransformer(model_name)

# Create the FAISS vector store
faiss_index = FAISS(embedding_function=embedding_model, index=index, docstore=None, index_to_docstore_id=None)

# Now you can use `faiss_index` as needed


In [ ]:
pip install transformers


GPT-Neo model used to generate responses

In [54]:
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer

# Load the GPT-Neo model and tokenizer
model_name = "EleutherAI/gpt-neo-1.3B"  # You can use "EleutherAI/gpt-neo-2.7B" if you have enough resources
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name)

# Set the pad token to be the eos token
tokenizer.pad_token = tokenizer.eos_token

def generate_response(query, retrieved_docs, max_input_length=1024, max_new_tokens=512):
    # Combine the query and retrieved documents
    input_text = f"Query: {query}\nDocuments:\n{retrieved_docs}\nAnswer:"

    # Tokenize the input
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=max_input_length)
    
    # Generate the response
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )
    
    # Decode the output
    response = tokenizer.decode(output[0], skip_special_tokens=True)
    return response

# Example usage
query = "What is the capital of France?"
# Combine the chunks of the top retrieved document as a single string
retrieved_docs = " ".join([chunk for chunk in processed_docs[:5]])  # Using the top 5 chunks as an example

response = generate_response(query, retrieved_docs)
print("Response:", response)


Response: Query: What is the capital of France?
Documents:
Document ID: 5733be284776f41900661182
Document ID: 5733be284776f41900661182
Chunk Number: 1
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary. Document ID: 5733be284776f4190066117f
Document ID: 5733be284776f4190066117f
Chunk Number: 1
Architecturally, the school has a Catholic character. Atop th